# Técnicas de Limpeza e Tratamento de Valores Ausentes para Análise de Dados.

## Pacotes Python usados no projeto

In [2]:
!pip install -q -U watermark

In [1]:
# imports
import math
import sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import warnings
warnings.filterwarnings('ignore')

In [2]:
%reload_ext watermark
%watermark -a "Leonardo da Silva Neves"

Author: Leonardo da Silva Neves



## Carregando os dados
https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

In [3]:
# Criando uma lista para identificar possíveis representações para valores ausentes
lista_labels_valores_ausentes = ["n/a", "na", "undefined"]

In [4]:
# Carregando o dataset com valores ausentes
# O parâmetro na_values identifica os argumentos de entrada como valores NaN. A ultilização
# deste parâmetro é valida quando se sabe de ante
dataset_dsa = pd.read_csv("dataset.csv", na_values= lista_labels_valores_ausentes)

In [5]:
# Shape
dataset_dsa.shape

(150001, 55)

In [6]:
# Alteração de parâmetro do Pandas "display.max_columns" para almentar o número de colunas exibidas ao imprimir o dataframe.
pd.set_option('display.max_columns', 100)
# Caso o df tenha mais de 100 colunaas, o que passar será trucado.

In [7]:
# Carregando o dicionário de dados
dicionario = pd.read_excel('dicionario.xlsx')

In [8]:
# Shape
dicionario.shape

(56, 2)

In [9]:
# Definindo um valor grande para a largura da coluna.
pd.set_option('display.max_colwidth', 100)

In [32]:
# Amostra de dados
# dicionario.head()

## Análise Exploratória

In [13]:
# info
dataset_dsa.info()

In [31]:
# Estatísticas descritivas
# dataset_dsa.describe()

Não faz sentido calcular estatísticas descritivas para variáveis Baread id, IMSI, MSISDN/Number e IMEI Embora essas colunas sejam
do tipo numérico, seus valores servem como identificadores. Mas o método describe() calcula as estatísticas de todas as colunas numéricas. Essas estatísticas estão sendo calculadas antes que os dados sejam limpos. Portanto, pode haver mudanças depois que os valores ausentes e outliers são tratados.

In [10]:
# Shape
dataset_dsa.shape

(150001, 55)

In [11]:
dicionario.shape

(56, 2)

Existem 150.001 linhas e 55 colunas no dataframe. No entando, temos 56 colunas com seus nomes e descrições no dicionário.
Isso significa que há uma coluna descrita, mas não incluída no dataframe.

In [14]:
# Concatenar os datasets para comparação de variáveis (lado a lado)
df_compara_colunas = pd.concat([pd.Series(dataset_dsa.columns.tolist()), dicionario['Fields']], axis=1)

In [18]:
df_compara_colunas.columns

Index(['Coluna do Dataset', 'Coluna do Dicionário'], dtype='object')

In [16]:
# Renomeia as colunas do df_compara_colunas
df_compara_colunas.rename(columns = {0:'Coluna do Dataset', 'Fields': 'Coluna do Dicionário'}, inplace = True)

In [30]:
# df_compara_colunas

"Dur.(Ms)" é ignorado no dataset como visto no indice 1 em df_compara_colunas. É ai que a ordem das colunas começa a mudar.
Mas o mesmo nome da coluna "Dur.(Ms)" aparece no dataser no indice 5, enquanto o arquivo de dicionário nos diz que é "Dur.(S)"
no indice 6. Como as medidas de ambas as colunas diferem conforme mostrado em seus nomes, nós precisamos verificar qual está
certo. Para investigar isso, usaremos a coluna "Dur.(Ms).1" que se encontra nos indices 28 e 29 no dataset e no arquivo de dicionário,
respectivamente.

In [29]:
# Filtro de colunas no df dataset_dsa para verificar se os valores das variáveis são identicos mas com
# o nome de coluna diferente. (duplicação de variável).
dataset_dsa[['Dur. (ms)', 'Dur. (ms).1']]

Parece que a coluna "Dur. (Ms)" é medida em segundos. Portanto, vamos renomeá-la apropriadamente. Vamos também renomear
algumas colunas para que fiquem claras como sua descrição e sigam o estilo de nomenclatura de outras colunas.

In [26]:
# Renomear as colunas do df dataset_dsa (parâmetro inplace = True para salvar no próprio df).
dataset_dsa.rename(columns= {'Dur. (ms)': 'Dur (s)',
                             'Dur. (ma).1': 'Dur (ms)',
                             'Start ms' : 'Start Offset (ms)',
                             'End ms': 'End Offset (ms)'},
                   inplace = True)

In [28]:
# Lista de colunas do dataset
dataset_dsa.shape

(150001, 55)

## Etapa 1 - Tratamento de Valores Ausentes

- 1- Identificando Valores Ausentes
- 2- Drop de Colunas
- 3- Imputação com Preenchimento Reverso
- 4- Imputação com Preenchimento Progressivo
- 5- Imputação de Variáceis Categóricas
- 6- Drop de Linhas